In [1]:
cd /Users/karolinegriesbach/Documents/Innkeepr/Git/evaluation-and-execution-scripts

In [2]:
import pandas as pd
from datetime import datetime, timedelta
from src.utils.prefect_api import call_prefect_api

result = pd.DataFrame(columns=["deployment_id","flow_id","start_time","state_name","deployment_name"])
query_columns = ["deployment_id","flow_id","start_time","state_name"]
to_date = datetime.now()
from_date = to_date - timedelta(weeks=4)
from_date_formatted = from_date.strftime("%Y-%m-%dT00:00:00Z")
to_date_formatted = to_date.strftime("%Y-%m-%dT23:59:59Z")
print(from_date_formatted, to_date_formatted)
path_save = "/Users/karolinegriesbach/Documents/Innkeepr/Git/evaluation-and-execution-scripts/SprintStories/EN-2961-saving-potentials/"
ignore_audiences = ["65f301e9f7b4d736196a0ea2",
"65c0f024f2e2b9b5a9ba2a4a",
"6617a6ed01c2ab3bd127f8c5",
"6687daf21767991e3bdfb414",
"6687da6e1767991e3bdfa9d8"]

In [3]:
all_deployments = call_prefect_api(
    endpoint = "/deployments/filter",
    json_data = {}
)
print(f"All deployments: {len(all_deployments)}")

In [4]:
for deployment in all_deployments:
    print(deployment["name"], deployment["id"])
    deployment_id = deployment["id"]
    deployment_name = deployment["name"]
    filter_payload = {
        "flow_runs": {
            "start_time": {
                "after_": from_date_formatted,
                "before_": to_date_formatted,
            }
        },
        "deployments": {
            "id": {"any_": [deployment_id]}
        }
    }
    response = call_prefect_api("/flow_runs/filter", filter_payload)
    if len(response)==0:
        print(".. No flows found")
        continue
    print(f".. found {len(response)} flows")
    response = pd.json_normalize(response)[query_columns]
    response["deployment_name"] = deployment["name"]
    result = pd.concat([result, response[result.columns]], ignore_index=True)
result

In [5]:
result = result.drop_duplicates(keep="first")
result.to_csv(f"{path_save}prefect_flows.csv")

In [6]:
result = pd.read_csv(f"{path_save}prefect_flows.csv")

In [7]:
result["deployment"] = result["deployment_name"].apply(lambda x: "-".join(x.split("-")[0:2]) if "googleConversionUpdate" not in x else x.split("-")[0])
result

In [8]:
for aud in ignore_audiences:
    print(result.shape)
    result = result[result["deployment_name"].str.contains(aud)==False]


In [9]:
result_states = result.groupby(by=["deployment","state_name"])[["deployment_id"]].count().reset_index()
result_states = result_states.pivot(index="deployment",columns="state_name",values="deployment_id")
result_states = result_states.sort_values(by="Failed",ascending=False).reset_index()
result_states["percentage_failed"] = result_states["Failed"] / (result_states["Failed"] + result_states["Completed"])
result_states[["deployment","Completed","Failed","percentage_failed"]]

In [10]:
result_states_name = result.groupby(by=["deployment_name","state_name"])[["deployment_id"]].count().reset_index()
result_states_name = result_states_name.pivot(index="deployment_name",columns="state_name",values="deployment_id")
result_states_name = result_states_name.sort_values(by="Failed",ascending=False).reset_index()
result_states_name["percentage_failed"] = result_states_name["Failed"] / (result_states_name["Failed"] + result_states_name["Completed"])
result_states_name[["deployment_name","Completed","Failed","percentage_failed"]]